7-1: Shapes of CNNs

Code.7-1-1: Shapes in the Feature Extraction

In [15]:
import tensorflow as tf

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

# need to constance
# random noise 32
N, n_H, n_W, n_C = 32, 28, 28, 3
n_conv_filters = 5
k_size = 3
pool_size, pool_strides = 2, 2
batch_size = 32

# Filers Extractor(Convolution + MaxPooling)
x = tf.random.normal(shape=(N, n_H, n_W, n_C))
conv1 = Conv2D(filters=n_conv_filters, kernel_size=k_size, padding = 'same', activation='relu')
conv1_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

conv2 = Conv2D(filters=n_conv_filters, kernel_size=k_size, padding='same', activation='relu')
conv2_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

flatten = Flatten()

# print result
print("Input: {}\n".format(x.shape))

x = conv1(x)
# Weight/Bias 
W, B = conv1.get_weights()
print("W/B: {}/{}".format(W.shape, B.shape))
print("After conv1: {}".format(x.shape))
x= conv1_pool(x)
print("After conv1_pool: {}".format(x.shape))
      
x = conv2(x)
W, B = conv2.get_weights()
print("W/B: {}/{}".format(W.shape, B.shape))
print("After conv2: {}".format(x.shape))
x= conv2_pool(x)
print("After conv2_pool: {}".format(x.shape))

x= flatten(x)
print("After Flatten: {}".format(x.shape))


Input: (32, 28, 28, 3)

W/B: (3, 3, 3, 5)/(5,)
After conv1: (32, 28, 28, 5)
After conv1_pool: (32, 14, 14, 5)
W/B: (3, 3, 5, 5)/(5,)
After conv2: (32, 14, 14, 5)
After conv2_pool: (32, 7, 7, 5)
After Flatten: (32, 245)


Code.7-1-2: Shapes in the Classifier

In [16]:
import tensorflow as tf

from tensorflow.keras.layers import Dense

# Activation function and Softmax
n_neurons = [50, 25, 10]
dense1 = Dense(units=n_neurons[0], activation='relu')
dense2 = Dense(units=n_neurons[1], activation='relu')
dense3 = Dense(units=n_neurons[2], activation='softmax')

# print result
print("Input features: P{}".format(x.shape))
x = dense1(x)
# Weight/Bias
W, B = dense1.get_weights()
print("W/B :{}/{}".format(W.shape, B.shape))
print("After dense1: {}\n".format(x.shape))

x = dense2(x)
W, B = dense2.get_weights()
print("W/B: {}/{}".format(W.shape, B.shape))
print("After dense2: {}\n".format(x.shape))

x = dense3(x)
W, B = dense3.get_weights()
print("W/B: {}/{}".format(W.shape, B.shape))
print("After dense2: {}".format(x.shape))

Input features: P(32, 245)
W/B :(245, 50)/(50,)
After dense1: (32, 50)

W/B: (50, 25)/(25,)
After dense2: (32, 25)

W/B: (25, 10)/(10,)
After dense2: (32, 10)


Code.7-1-3: Shapes in the Loss Functions

In [20]:
from tensorflow.keras.losses import CategoricalCrossentropy

y = tf.random.uniform(minval=0, maxval=10,
                      shape=(32, ),
                      dtype=tf.int32)
y = tf.one_hot(y, depth=10)

loss_object = CategoricalCrossentropy()
loss = loss_object(y, x)
print(loss.shape)
print(loss)


()
tf.Tensor(2.3129845, shape=(), dtype=float32)


7-1-2: Implementation of CNNs

Code.7-1-4: Implementaion with Sequential Method

In [24]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
k_size, padding = 3, 'same'
activation = 'relu'
pool_size, pool_strides= 2, 2

x= tf.random.normal(shape=(N, n_H, n_W, n_C))
print(x.shape)

model = Sequential()
model.add(Conv2D(filters=n_conv_neurons[0], kernel_size=k_size, padding=padding,
                 activation=activation))

model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))
model.add(Conv2D(filters=n_conv_neurons[1], kernel_size=k_size, padding=padding,
                 activation=activation))

model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))
model.add(Conv2D(filters=n_conv_neurons[2], kernel_size=k_size, padding=padding,
                 activation=activation))
model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))
model.add(Flatten())

model.add(Dense(units=n_dense_neurons[0], activation=activation))
model.add(Dense(units=n_dense_neurons[1], activation=activation))
model.add(Dense(units=n_dense_neurons[2], activation='softmax'))

predictions = model(x)
print(predictions.shape)




(4, 28, 28, 3)
(4, 10)


In [25]:
model = Sequential()
for n_conv_neurons in n_conv_neurons:
    model.add(Conv2D(filters=n_conv_neurons, kernel_size=k_size, padding=padding,
                     activation=activation))
    model.add(MaxPooling2D(pool_size=pool_size, strides=pool_strides))
model = Flatten()

for n_dense_neurons in n_dense_neurons:
    model.add(Dense(units=n_dense_neurons, activation=activation))
model.add(Dense(units=n_dense_neurons[-1], activation=activation))


AttributeError: 'Flatten' object has no attribute 'add'

Code.7-1-5: Implementation with Model Sub-classing

In [36]:
import tensorflow as tf

from tensorflow.keras.models import Model

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

N, n_H, n_W, n_C = 4, 28, 28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
k_size, padding = 3, 'same'
activation= 'relu'
pool_size, pool_strides= 2, 2

x= tf.random.normal(shape=(N, n_H, n_W, n_C))

class TestCNN(Model):
  def __init__(self):
    super(TestCNN, self).__init__()

    self.conv1= Conv2D(filters=n_conv_neurons[0], kernel_size=k_size, padding=padding,
                       activation=activation)
    self.conv1_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)
    
    self.conv2= Conv2D(filters=n_conv_neurons[1], kernel_size=k_size, padding=padding,
                       activation=activation)
    self.conv2_pool= MaxPooling2D(pool_size=pool_size, strides=pool_strides)
    
    self.conv3= Conv2D(filters=n_conv_neurons[2], kernel_size=k_size, padding=padding,
                       activation=activation)
    self.conv3_pool= MaxPooling2D(pool_size=pool_size, strides=pool_strides)
    self.flatten= Flatten()

    self.dense1= Dense(units=n_dense_neurons[0], activation=activation)
    self.dense2= Dense(units=n_dense_neurons[1], activation=activation)
    self.dense3= Dense(units=n_dense_neurons[2], activation='softmax')

  def call(self, x):
    x =self.conv1(x)
    print(x.shape)
    x =self.conv1_pool(x)
    print(x.shape)

    x = self.conv2(x)
    print(x.shape)
    x = self.conv2_pool(x)
    print(x.shape)

    x = self.conv3(x)
    print(x.shape)
    x = self.conv3_pool(x)
    print(x.shape)
      
    x = self.flatten(x)
    print(x.shape)

    x = self.dense1(x)
    print(x.shape)
    x = self.dense2(x)
    print(x.shape)
    x = self.dense3(x)
    print(x.shape)
    return x

N, n_H, n_W, n_C= 4, 28,28, 3
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
k_size, padding = 3, 'same'
pool_size, pool_strides = 2, 2

x= tf.random.normal(shape=(N, n_H, n_W, n_C))

model = TestCNN()
y = model(x)

    


(4, 28, 28, 10)
(4, 14, 14, 10)
(4, 14, 14, 20)
(4, 7, 7, 20)
(4, 7, 7, 30)
(4, 3, 3, 30)
(4, 270)
(4, 50)
(4, 30)
(4, 10)


Code.7-1-6: Implementation with Sequential + Layer Sub-classing

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class MyConv(Layer):
    def __init__(self, n_neurons):
        super(MyConv, self).__init__()

        self.conv= Conv2D(filters=n_neurons, kernel_size=k_size, padding=padding,
                          activation=activation)
        self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

    def call(self, x):
        x = self.conv(x)
        x = self.conv_pool(x)
        return x



Code.7-1-7: Implementation with Model and Layer Sub-classing

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class MyConv(Layer):
    def __init__(self, n_neurons):
        super(MyConv, self).__init__()

        self.conv = Conv2D(fiters=n_neurons, kernel_size=k_size, padding=padding,
                           activation=activation)
        
        self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

    def call(self, x):
        x = self.conv(x)
        x = self.conv_pool(x)
        return x
    
class TestCNN(Model):
    def __init__(self):
        super(TestCNN, self).__init__()

        self.conv1 = MyConv(n_conv_neurons[0])
        self.conv2 = MyConv(n_conv_neurons[1])
        self.conv3 = MyConv(n_conv_neurons[2])
        self.flatten = Flatten()

        self.dense = Dense(units=n_neurons[0], activation=activation)
        self.dense2 = Dense(units=n_neurons[1], activation=activation)
        self.dense3 = Dense(units=n_neurons[2], activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        
        x = self.dense(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class MyConv(Layer):
    def __init__(self, n_neurons):
        super(MyConv, self).__init__()

        self.conv= Conv2D(filters=n_neurons, kernel_size=k_size, padding=padding,
                          activation=activation)
        self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)

    def call(self, x):
        x = self.conv(x)
        x = self.conv_pool(x)
        return x
    
class TestCNN(Model):
    def __init__(self):
        super(TestCNN, self).__init__()
        self.fe = Sequential

        self.fe_add(MyConv(n_conv_neurons[0]))
        self.fe_add(MyConv(n_conv_neurons[1]))
        self.fe_add(MyConv(n_conv_neurons[2]))

        self.classifier = Sequential
        self.classifier.add(Dense(units=n_dense_neurons[0], activation=activation))
        self.classifier.add(Dense(units=n_dense_neurons[1], activation=activation))
        self.classifier.add(Dense(units=n_dense_neurons[2], activation='softmax'))

    def call(self, x):
        self.fe_add(x)

        self.classifier(x)
        return x        

7-2: Implementation of LeNet

Code.7-2-1: LeNet with Hybrid Sub-classing

In [43]:
import tensorflow as tf

from tensorflow.keras.models import Model

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class LeNet(Model):
    def __init__(self):
        super(LeNet, self).__init__()

        self.conv1 = Conv2D(filters=6, kernel_size=5, padding ='same',
                            activation='tanh')
        self.conv1_pool = AveragePooling2D(pool_size=2, strides=2)

        self.conv2 = Conv2D(filters=16, kernel_size=5, padding='valid',
                        activation='tanh')
        self.conv2_pool = AveragePooling2D(pool_size=2, strides=2)

        self.conv3 = Conv2D(filters=120, kernel_size= 5, padding='valid',
                            activation= 'tanh')
        self.flatten = Flatten()

        self.dense1 = Dense(units=84, activation='tanh')
        self.dense2 = Dense(units=10, activation='softmax')

    def call(self, x):
        print("x: {}".format(x.shape))

        x =self.conv1(x)
        print("x: {}".format(x.shape))
        x = self.conv1_pool(x)
        print("x: {}".format(x.shape))

        x = self.conv2(x)
        print("x: {}".format(x.shape))
        x = self.conv2_pool(x)
        print("x: {}".format(x.shape))

        x = self.conv3(x)
        print("x: {}".format(x.shape))
        x = self.flatten(x)
        print("x: {}".format(x.shape))

        x = self.dense1(x)
        print("x: {}".format(x.shape))
        x = self.dense2(x)
        print("x: {}".format(x.shape))
        return x

model = LeNet()

x = tf.random.normal(shape=(32, 28, 28, 1))
predictions = model(x)

x: (32, 28, 28, 1)
x: (32, 28, 28, 6)
x: (32, 14, 14, 6)
x: (32, 10, 10, 16)
x: (32, 5, 5, 16)
x: (32, 1, 1, 120)
x: (32, 120)
x: (32, 84)
x: (32, 10)


Code.7-2-2: LeNet with Hybrid Method

In [57]:
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class ConvLayer(Layer):
    def __init__(self, filters, padding, pool=True):
        super(ConvLayer, self).__init__()
        self.pool = pool

        self.conv = Conv2D(filters=6, kernel_size=5, padding='same',
                            activation='tanh')

        if self.pool == True:
            self.conv_pool = AveragePooling2D(pool_size=2, strides=2)

    def call(self,x):
      x = self.conv(x)
      if self.pool == True:
         x = self.conv_pool(x)
      return x

class LeNet(Model):
    def __init__(self):
      super(LeNet, self).__init__()
      self.convl = ConvLayer(filters=6, padding='same')
      self.conv2 = ConvLayer(filters=16, padding='valid')
      self.conv3 = ConvLayer(filters=120, padding='valid', pool=False)
      self.flatten = Flatten()

      self.dense1 = Dense(units=84, activation='tanh')
      self.dense2 = Dense(units=10, activation='softmax')

    def call(self, x):
      x = self.convl(x)
      x = self.conv2(x)
      x = self.conv3(x)

      x = self.flatten(x)
      x = self.dense1(x)
      x = self.dense2(x)
      return x

model = LeNet()

x = tf.random.normal(shape=(32, 28, 28, 1))
prediction = model(x)

Code.7-2-3: Forward Propagation of LeNet

In [66]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

from tensorflow.keras.datasets import mnist

from tensorflow.keras.losses import SparseCategoricalCrossentropy

########LeNet Implementation#########
class ConvLayer(Layer):
  def __init__ (self, filters, padding, pool=True):
    super(ConvLayer, self).__init__()
    self.pool = pool
    self.conv = Conv2D(filters=6, kernel_size=5, padding='same',
                      activation='tanh')
    
    if self.pool == True:
      self.conv_pool = AveragePooling2D(pool_size=2, strides=2)

    def call(self,x):
      x = self.conv(x)
      if self.pool == True:
        x = self.conv_pool(x)
      return x

class LeNet (Model):
  def __init__(self):
      super(LeNet, self).__init__()
      self.convl = ConvLayer(filters=6, padding='same')
      self.conv2 = ConvLayer(filters=16, padding='valid')
      self.conv3 = ConvLayer(filters=120, padding='valid', pool=False)
      self.flatten = Flatten()

      self.dense1 = Dense(units=84, activation='tanh')
      self.dense2 = Dense(units=10, activation ='softmax')

  def call(self, x):
      x = self.convl(x)
      x = self.conv2(x)
      x = self.conv3(x)
      x = self.flatten(x)
      x = self.dense1(x)
      x = self.dense2(x)
      return x


model = LeNet()

x = tf.random.normal(shape=(32, 28, 28, 1))
predictions = model(x)

######Dataset Perparation#####
(train_images, train_labels), _ = mnist. load_data()
train_images = np.expand_dims(train_images, axis=3).astype(np.float32)
train_labels = train_labels.astype(np. int32)     
              
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.batch(32)

#####Forward Propagation######
model = LeNet()
loss_object = SparseCategoricalCrossentropy()

for images, labels in train_ds:
    predictions = model(images)
    loss = loss_object(labels, predictions)
    print(loss)
    break


tf.Tensor(2.7464359, shape=(), dtype=float32)
